# FLFP Models

## Setup

In [130]:
import pandas as pd
import numpy as np

# For data splitting and preprocessing
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.preprocessing import StandardScaler

# For modeling and evaluation
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

## Data preparation

### Load the FLFP dataset

In [131]:
flfp_df = pd.read_parquet('data/flfp_dataset.parquet')
flfp_df['region'] = flfp_df['region'].str.strip()  # Clean trailing spaces
flfp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5208 entries, 0 to 5207
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country_name             5208 non-null   object 
 1   flfp_15_64               4484 non-null   float64
 2   year                     5208 non-null   int64  
 3   fertility_rate           5208 non-null   float64
 4   fertility_adolescent     5208 non-null   float64
 5   urban_population         5160 non-null   float64
 6   dependency_ratio         5208 non-null   float64
 7   life_exp_female          5208 non-null   float64
 8   infant_mortality         4704 non-null   float64
 9   population_total         5208 non-null   float64
 10  secondary_enroll_fe      3427 non-null   float64
 11  tertiary_enroll_fe       2979 non-null   float64
 12  gender_parity_primary    2874 non-null   float64
 13  gender_parity_secondary  2934 non-null   float64
 14  gdp_per_capita_const    

### Perform train-test split
With this being panel data, we have to be careful not to leak future data for any country into the training set. We could try a time-base split, but time-series forecasting is not necessarily our goal here. Instead, we'll do a country-based split, ensuring that all years for a given country are either in the training or test set.

**Note on stratification**: We initially attempted to stratify the split by region or income level to ensure balanced representation across splits. However, this approach did not improve model performance and in some cases led to worse validation metrics, likely due to the relatively small number of countries per category creating unlucky splits. We therefore use a simple random split with a fixed seed for reproducibility, which empirically produces reasonable balance across most key dimensions.

In [132]:
# Filter to FLFP observations
modeling_df = flfp_df[flfp_df['flfp_15_64'].notna()].copy()

print("Country-based Train/Validation/Test Split")
print("=" * 50)

# Get unique countries and their observation counts
country_stats = modeling_df.groupby('country_name').size().reset_index(name='obs_count')
country_stats = country_stats.sort_values('obs_count', ascending=False)

print(f"Total observations: {len(modeling_df):,}")
print(f"Unique countries: {len(country_stats)}")
print(f"Obs per country - Min: {country_stats['obs_count'].min()}, Max: {country_stats['obs_count'].max()}, Mean: {country_stats['obs_count'].mean():.1f}")

# Step 1: Split countries into train (60%) and temp (40%)
# (Crucially, we are splitting country names into sets, not individual observations)
countries_train, countries_temp = train_test_split(
    country_stats['country_name'].tolist(),
    test_size=0.4,  # 40% for val + test
    random_state=42
)

# Step 2: Split temp into validation (20%) and test (20%)
countries_val, countries_test = train_test_split(
    countries_temp,
    test_size=0.5,  # 50% of 40% = 20% overall
    random_state=42
)

print(f"\nCountries in training set: {len(countries_train)} ({len(countries_train)/len(country_stats)*100:.1f}%)")
print(f"Countries in validation set: {len(countries_val)} ({len(countries_val)/len(country_stats)*100:.1f}%)")
print(f"Countries in test set: {len(countries_test)} ({len(countries_test)/len(country_stats)*100:.1f}%)")

# Create train/val/test datasets based on country assignment
# (Here we are creating boolean masks to identify which observations belong to which set)
train_mask = modeling_df['country_name'].isin(countries_train)
val_mask = modeling_df['country_name'].isin(countries_val)
test_mask = modeling_df['country_name'].isin(countries_test)

# Subset the main dataframe
train_df = modeling_df[train_mask].copy()
val_df = modeling_df[val_mask].copy()
test_df = modeling_df[test_mask].copy()

print(f"\nTraining observations: {len(train_df):,} ({len(train_df)/len(modeling_df)*100:.1f}%)")
print(f"Validation observations: {len(val_df):,} ({len(val_df)/len(modeling_df)*100:.1f}%)")
print(f"Test observations: {len(test_df):,} ({len(test_df)/len(modeling_df)*100:.1f}%)")

# Verify no country overlap
train_countries_set = set(train_df['country_name'])
val_countries_set = set(val_df['country_name'])
test_countries_set = set(test_df['country_name'])

assert len(train_countries_set & val_countries_set) == 0, "Train/Val country overlap detected!"
assert len(train_countries_set & test_countries_set) == 0, "Train/Test country overlap detected!"
assert len(val_countries_set & test_countries_set) == 0, "Val/Test country overlap detected!"

print("\n✓ Verified: No countries appear in multiple sets")

# Preview country assignments
print(f"\nSample training countries: {countries_train[:5]}")
print(f"Sample validation countries: {countries_val[:3]}")
print(f"Sample test countries: {countries_test[:3]}")

# Store the split for later use
country_split = {
    'train_countries': countries_train,
    'val_countries': countries_val, 
    'test_countries': countries_test,
    'train_df': train_df,
    'val_df': val_df,
    'test_df': test_df
}

Country-based Train/Validation/Test Split
Total observations: 4,484
Unique countries: 187
Obs per country - Min: 22, Max: 24, Mean: 24.0

Countries in training set: 112 (59.9%)
Countries in validation set: 37 (19.8%)
Countries in test set: 38 (20.3%)

Training observations: 2,685 (59.9%)
Validation observations: 888 (19.8%)
Test observations: 911 (20.3%)

✓ Verified: No countries appear in multiple sets

Sample training countries: ['India', 'Nicaragua', 'Lesotho', 'Cameroon', 'New Caledonia']
Sample validation countries: ['Kuwait', 'Gabon', 'Burundi']
Sample test countries: ['Dominican Republic', 'Virgin Islands (U.S.)', 'Kenya']


### Add categorical features (region and income level)


In [133]:
# Filter out 'Not classified' income observations
print("Filtering out 'Not classified' income observations...")
initial_train = len(train_df)
initial_val = len(val_df)
initial_test = len(test_df)

train_df = train_df[train_df['income_level'] != 'Not classified'].copy()
val_df = val_df[val_df['income_level'] != 'Not classified'].copy()
test_df = test_df[test_df['income_level'] != 'Not classified'].copy()

print(f"  Train: {initial_train} → {len(train_df)} (-{initial_train - len(train_df)})")
print(f"  Val: {initial_val} → {len(val_df)} (-{initial_val - len(val_df)})")
print(f"  Test: {initial_test} → {len(test_df)} (-{initial_test - len(test_df)})")

# Create label-encoded income for tree-based models
print("\nCreating label-encoded income (for tree-based models)...")
income_mapping = {
    'Low income': 0,
    'Lower middle income': 1,
    'Upper middle income': 2,
    'High income': 3
}

train_df['income_level_encoded'] = train_df['income_level'].map(income_mapping)
val_df['income_level_encoded'] = val_df['income_level'].map(income_mapping)
test_df['income_level_encoded'] = test_df['income_level'].map(income_mapping)

print(f"  Income encoding: {income_mapping}")

# Create one-hot encoded region with clean names
print("\nCreating one-hot encoded region with clean names...")

# Region name mapping to clean snake_case abbreviations
region_name_mapping = {
    'East Asia & Pacific': 'region_eap',
    'Europe & Central Asia': 'region_eca',
    'Latin America & Caribbean': 'region_lac',
    'Middle East, North Africa, Afghanistan & Pakistan': 'region_mena_afpak',
    'North America': 'region_namerica',
    'South Asia': 'region_sasia',
    'Sub-Saharan Africa': 'region_ssa'
}

# Create dummies for ALL regions (drop_first=False to get all categories)
region_dummies_train = pd.get_dummies(train_df['region'], prefix='region', drop_first=False)
region_dummies_val = pd.get_dummies(val_df['region'], prefix='region', drop_first=False)
region_dummies_test = pd.get_dummies(test_df['region'], prefix='region', drop_first=False)

# Rename columns using mapping
for original, clean in region_name_mapping.items():
    old_col = f'region_{original}'
    if old_col in region_dummies_train.columns:
        region_dummies_train.rename(columns={old_col: clean}, inplace=True)
    if old_col in region_dummies_val.columns:
        region_dummies_val.rename(columns={old_col: clean}, inplace=True)
    if old_col in region_dummies_test.columns:
        region_dummies_test.rename(columns={old_col: clean}, inplace=True)

# Drop reference category: region_mena_afpak
reference_region = 'region_mena_afpak'
if reference_region in region_dummies_train.columns:
    region_dummies_train.drop(columns=[reference_region], inplace=True)
if reference_region in region_dummies_val.columns:
    region_dummies_val.drop(columns=[reference_region], inplace=True)
if reference_region in region_dummies_test.columns:
    region_dummies_test.drop(columns=[reference_region], inplace=True)

# Ensure all sets have the same columns (in case some regions are missing in val/test)
all_region_cols = region_dummies_train.columns.tolist()
for col in all_region_cols:
    if col not in region_dummies_val.columns:
        region_dummies_val[col] = 0
    if col not in region_dummies_test.columns:
        region_dummies_test[col] = 0

# Reorder columns to match
region_dummies_val = region_dummies_val[all_region_cols]
region_dummies_test = region_dummies_test[all_region_cols]

# Add to dataframes
train_df = pd.concat([train_df, region_dummies_train], axis=1)
val_df = pd.concat([val_df, region_dummies_val], axis=1)
test_df = pd.concat([test_df, region_dummies_test], axis=1)

print(f"  Created {len(all_region_cols)} region dummy variables")
print(f"  Reference category: {reference_region} (dropped)")
print(f"  Region columns: {all_region_cols}")

# Create one-hot encoded income with clean names (for linear models)
print("\nCreating one-hot encoded income with clean names (for linear models)...")

# Income level name mapping to clean snake_case abbreviations
income_name_mapping = {
    'High income': 'income_high',
    'Low income': 'income_low',
    'Lower middle income': 'income_lower_mid',
    'Upper middle income': 'income_upper_mid'
}

# Create dummies for ALL income levels (drop_first=False to get all categories)
income_dummies_train = pd.get_dummies(train_df['income_level'], prefix='income', drop_first=False)
income_dummies_val = pd.get_dummies(val_df['income_level'], prefix='income', drop_first=False)
income_dummies_test = pd.get_dummies(test_df['income_level'], prefix='income', drop_first=False)

# Rename columns using mapping
for original, clean in income_name_mapping.items():
    old_col = f'income_{original}'
    if old_col in income_dummies_train.columns:
        income_dummies_train.rename(columns={old_col: clean}, inplace=True)
    if old_col in income_dummies_val.columns:
        income_dummies_val.rename(columns={old_col: clean}, inplace=True)
    if old_col in income_dummies_test.columns:
        income_dummies_test.rename(columns={old_col: clean}, inplace=True)

# Drop reference category: income_low
reference_income = 'income_low'
if reference_income in income_dummies_train.columns:
    income_dummies_train.drop(columns=[reference_income], inplace=True)
if reference_income in income_dummies_val.columns:
    income_dummies_val.drop(columns=[reference_income], inplace=True)
if reference_income in income_dummies_test.columns:
    income_dummies_test.drop(columns=[reference_income], inplace=True)

# Ensure all sets have the same columns
all_income_cols = income_dummies_train.columns.tolist()
for col in all_income_cols:
    if col not in income_dummies_val.columns:
        income_dummies_val[col] = 0
    if col not in income_dummies_test.columns:
        income_dummies_test[col] = 0

# Reorder columns to match
income_dummies_val = income_dummies_val[all_income_cols]
income_dummies_test = income_dummies_test[all_income_cols]

# Add to dataframes
train_df = pd.concat([train_df, income_dummies_train], axis=1)
val_df = pd.concat([val_df, income_dummies_val], axis=1)
test_df = pd.concat([test_df, income_dummies_test], axis=1)

print(f"  Created {len(all_income_cols)} income dummy variables")
print(f"  Reference category: {reference_income} (dropped)")
print(f"  Income columns: {all_income_cols}")

print("\n✓ Categorical features added successfully!")
print(f"  Train shape: {train_df.shape}")
print(f"  Val shape: {val_df.shape}")
print(f"  Test shape: {test_df.shape}")


Filtering out 'Not classified' income observations...
  Train: 2685 → 2661 (-24)
  Val: 888 → 888 (-0)
  Test: 911 → 887 (-24)

Creating label-encoded income (for tree-based models)...
  Income encoding: {'Low income': 0, 'Lower middle income': 1, 'Upper middle income': 2, 'High income': 3}

Creating one-hot encoded region with clean names...
  Created 6 region dummy variables
  Reference category: region_mena_afpak (dropped)
  Region columns: ['region_eap', 'region_eca', 'region_lac', 'region_namerica', 'region_sasia', 'region_ssa']

Creating one-hot encoded income with clean names (for linear models)...
  Created 3 income dummy variables
  Reference category: income_low (dropped)
  Income columns: ['income_high', 'income_lower_mid', 'income_upper_mid']

✓ Categorical features added successfully!
  Train shape: (2661, 35)
  Val shape: (888, 35)
  Test shape: (887, 35)


### Preprocess the data
We need three datasets for different model types: raw data for tree-based models for XGBoost and LightGBM, imputed but unscaled for random forest, and imputed and scaled for linear models, SVM and KNN. 

For imputation, we will take the country median as the default strategy but if the country has all missing values for a feature, we will use the global mean instead.

For scaling, we will use standard scaling (mean=0, std=1) based on the training set statistics.

Note that we are conducting the imputation and scaling after splitting to avoid data leakage (in other words, using information from the test set to inform the training set transformations).

In [134]:
# Define base predictor columns (numeric features)
# Note: Removed unemployment_total, unemployment_female, and labor_force_total
# to avoid data leakage (these are mathematically related to FLFP)
base_predictor_cols = [
    'fertility_rate', 'fertility_adolescent', 'urban_population',
    'dependency_ratio', 'life_exp_female', 'infant_mortality',
    'population_total', 'secondary_enroll_fe', 'gdp_per_capita_const',
    'gdp_growth', 'services_gdp', 'industry_gdp', 'rule_of_law'
 ]

# Get the categorical feature column names that were created in the previous cell
region_cols = [col for col in train_df.columns if col.startswith('region_')]
# Get income dummy columns (exclude 'income_level' and 'income_level_encoded')
income_dummy_cols = [col for col in train_df.columns if col.startswith('income_') and col not in ['income_level', 'income_level_encoded']]

# Create separate predictor lists for different model types
# Tree-based models: use label-encoded income + one-hot region
predictor_cols_tree = base_predictor_cols + ['income_level_encoded'] + region_cols

# Linear models: use one-hot encoded income + one-hot region
predictor_cols_linear = base_predictor_cols + income_dummy_cols + region_cols

print("Predictor Column Setup:")
print(f"  Base numeric features: {len(base_predictor_cols)}")
print(f"  Region dummies: {len(region_cols)}")
print(f"  Income dummies: {len(income_dummy_cols)}")
print(f"  Total for tree models: {len(predictor_cols_tree)} (base + income_encoded + regions)")
print(f"  Total for linear models: {len(predictor_cols_linear)} (base + income_dummies + regions)")

target_col = 'flfp_15_64'

# Variables that need imputation (only numeric features need imputation)
variables_to_impute = [
    'secondary_enroll_fe', 'urban_population', 'infant_mortality',
    'gdp_per_capita_const', 'gdp_growth', 'services_gdp',
    'industry_gdp', 'rule_of_law'
 ]

def panel_imputation(train_df, val_df, test_df, variables_to_impute):
    """Apply country-specific median imputation without data leakage"""

    # Calculate imputation rules using ONLY training data
    train_country_medians = {}
    train_year_medians = {}
    train_global_medians = {}

    for var in variables_to_impute:
        # Country-specific medians from training data only
        train_country_medians[var] = train_df.groupby('country_name')[var].median()
        # Year-specific medians from training data only
        train_year_medians[var] = train_df.groupby('year')[var].median()
        # Global median from training data only
        train_global_medians[var] = train_df[var].median()

    # Apply imputation rules to all datasets
    def apply_imputation(df):
        df_imputed = df.copy()
        for var in variables_to_impute:
            if var in df_imputed.columns:
                # Use training-based country medians
                for country in df_imputed['country_name'].unique():
                    country_mask = df_imputed['country_name'] == country
                    country_median = train_country_medians[var].get(country, np.nan)

                    # Fill using country median where available
                    df_imputed.loc[country_mask, var] = df_imputed.loc[country_mask, var].fillna(country_median)

                # Fill remaining NaNs using year medians
                year_values = df_imputed.loc[:, 'year']
                missing_mask = df_imputed[var].isna()
                if missing_mask.any():
                    years_to_fill = year_values[missing_mask]
                    fill_values = years_to_fill.map(train_year_medians[var]).astype(float)
                    df_imputed.loc[missing_mask, var] = fill_values

                # Fall back to training-global median if still missing
                df_imputed[var] = df_imputed[var].fillna(train_global_medians[var])

        return df_imputed

    return apply_imputation(train_df), apply_imputation(val_df), apply_imputation(test_df)

# Log-transform population_total in the original dataframes (before imputation)
# This will affect both raw and imputed datasets
print("\nApplying log transformation to population_total (before extraction)...")
train_df['population_total'] = np.log(train_df['population_total'])
val_df['population_total'] = np.log(val_df['population_total'])
test_df['population_total'] = np.log(test_df['population_total'])
print("  ✓ Log transformation applied to raw data")

# Extract TRULY raw features (before imputation) for models that handle missing values natively
# Use tree predictor columns (includes income_level_encoded + region dummies)
X_train_raw = train_df[predictor_cols_tree].copy()
X_val_raw = val_df[predictor_cols_tree].copy()
X_test_raw = test_df[predictor_cols_tree].copy()

print("\nDataset 1 - Truly Raw/Unimputed (XGBoost, LightGBM):")
print(f"  Using predictor_cols_tree ({len(predictor_cols_tree)} features)")
print(f"  X_train_raw: {X_train_raw.shape}")
print(f"  X_val_raw: {X_val_raw.shape}")
print(f"  X_test_raw: {X_test_raw.shape}")
print(f"  Missing values - Train: {X_train_raw.isna().sum().sum()}")
print(f"  Missing values - Val: {X_val_raw.isna().sum().sum()}")
print(f"  Missing values - Test: {X_test_raw.isna().sum().sum()}")

# Apply imputation
train_clean, val_clean, test_clean = panel_imputation(
    train_df, val_df, test_df, variables_to_impute
)

# Extract imputed features and target
# Use tree predictor columns (includes income_level_encoded + region dummies)
X_train_imputed = train_clean[predictor_cols_tree].copy()
X_val_imputed = val_clean[predictor_cols_tree].copy()
X_test_imputed = test_clean[predictor_cols_tree].copy()

y_train = train_clean[target_col].copy()
y_val = val_clean[target_col].copy()
y_test = test_clean[target_col].copy()

print("\nImputation complete:")
print("Dataset 2 - Imputed (Random Forest):")
print(f"  Using predictor_cols_tree ({len(predictor_cols_tree)} features)")
print(f"  X_train_imputed: {X_train_imputed.shape}")
print(f"  X_val_imputed: {X_val_imputed.shape}")
print(f"  X_test_imputed: {X_test_imputed.shape}")
print(f"  Missing values - Train: {X_train_imputed.isna().sum().sum()}")
print(f"  Missing values - Val: {X_val_imputed.isna().sum().sum()}")
print(f"  Missing values - Test: {X_test_imputed.isna().sum().sum()}")

# Create scaled versions (fit scaler only on training data)
# Use linear predictor columns (includes income dummies + region dummies)
print("\nCreating scaled datasets...")
scaler = StandardScaler()

# Need to extract linear predictor columns from the clean dataframes
X_train_linear = train_clean[predictor_cols_linear].copy()
X_val_linear = val_clean[predictor_cols_linear].copy()
X_test_linear = test_clean[predictor_cols_linear].copy()

X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_linear),
    columns=predictor_cols_linear,
    index=X_train_linear.index
)
X_val_scaled = pd.DataFrame(
    scaler.transform(X_val_linear),
    columns=predictor_cols_linear,
    index=X_val_linear.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_linear),
    columns=predictor_cols_linear,
    index=X_test_linear.index
)

print("\nDataset 3 - Scaled + Imputed (Linear models, SVM, KNN):")
print(f"  Using predictor_cols_linear ({len(predictor_cols_linear)} features)")
print(f"  X_train_scaled: {X_train_scaled.shape}")
print(f"  X_val_scaled: {X_val_scaled.shape}")
print(f"  X_test_scaled: {X_test_scaled.shape}")

print("\nTarget variable (same for all models):")
print(f"  y_train: {y_train.shape}")
print(f"  y_val: {y_val.shape}")
print(f"  y_test: {y_test.shape}")

# Create GroupKFold for hyperparameter tuning to prevent data leakage
# This ensures all observations from the same country stay in the same fold
print("\nSetting up GroupKFold cross-validation...")
groups_train = train_clean['country_name'].values
group_kfold = GroupKFold(n_splits=5)
print(f"  Using GroupKFold with 5 splits grouped by country")
print(f"  This prevents different years from the same country appearing in different folds")

print("\n✓ All datasets ready for modeling!")
print(f"Tree models: {len(predictor_cols_tree)} features (numeric + income_encoded + region_dummies)")
print(f"Linear models: {len(predictor_cols_linear)} features (numeric + income_dummies + region_dummies)")
print(f"Total observations: {len(X_train_raw) + len(X_val_raw) + len(X_test_raw):,}")

Predictor Column Setup:
  Base numeric features: 13
  Region dummies: 6
  Income dummies: 3
  Total for tree models: 20 (base + income_encoded + regions)
  Total for linear models: 22 (base + income_dummies + regions)

Applying log transformation to population_total (before extraction)...
  ✓ Log transformation applied to raw data

Dataset 1 - Truly Raw/Unimputed (XGBoost, LightGBM):
  Using predictor_cols_tree (20 features)
  X_train_raw: (2661, 20)
  X_val_raw: (888, 20)
  X_test_raw: (887, 20)
  Missing values - Train: 1903
  Missing values - Val: 368
  Missing values - Test: 329

Imputation complete:
Dataset 2 - Imputed (Random Forest):
  Using predictor_cols_tree (20 features)
  X_train_imputed: (2661, 20)
  X_val_imputed: (888, 20)
  X_test_imputed: (887, 20)
  Missing values - Train: 0
  Missing values - Val: 0
  Missing values - Test: 0

Creating scaled datasets...

Dataset 3 - Scaled + Imputed (Linear models, SVM, KNN):
  Using predictor_cols_linear (22 features)
  X_train_sca

## Linear Regression

### Simple OLS (no regularization)

In [135]:
# Initialize the model
ols_model = LinearRegression()

# Fit on training data (using scaled data for linear models)
ols_model.fit(X_train_scaled, y_train)

# Make predictions on train and validation sets
y_train_pred = ols_model.predict(X_train_scaled)
y_val_pred = ols_model.predict(X_val_scaled)

# Calculate performance metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Display results
print(f"\nTraining Performance:")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE:  {train_mae:.3f}")
print(f"  R²:   {train_r2:.3f}")

print(f"\nValidation Performance:")
print(f"  RMSE: {val_rmse:.3f}")
print(f"  MAE:  {val_mae:.3f}")
print(f"  R²:   {val_r2:.3f}")

# Feature importance for OLS (coefficient magnitudes)
feature_importance = pd.DataFrame({
    'feature': predictor_cols_linear,
    'importance': np.abs(ols_model.coef_)
}).sort_values('importance', ascending=False)

print(f"\nTop 10 Most Important Features:")
print(feature_importance.head(10))


Training Performance:
  RMSE: 11.005
  MAE:  8.400
  R²:   0.531

Validation Performance:
  RMSE: 12.045
  MAE:  9.660
  R²:   0.462

Top 10 Most Important Features:
                 feature  importance
17            region_eca   13.607349
16            region_eap   12.771008
21            region_ssa   11.205452
18            region_lac    9.671223
4        life_exp_female    8.629168
13           income_high    8.126479
3       dependency_ratio    7.568505
15      income_upper_mid    7.103070
1   fertility_adolescent    6.010921
8   gdp_per_capita_const    5.693085


### Lasso Regression (L1 regularization)

In [136]:
# Define alpha values to test (regularization strength)
alpha_values = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

# Initialize Lasso with GridSearch for hyperparameter tuning
lasso_grid = GridSearchCV(
    Lasso(random_state=42, max_iter=2000),
    param_grid={'alpha': alpha_values},
    cv=group_kfold,  # Use GroupKFold to prevent country leakage
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Fit on training data (using scaled data)
lasso_grid.fit(X_train_scaled, y_train, groups=groups_train)

# Get the best model
lasso_model = lasso_grid.best_estimator_
best_alpha = lasso_grid.best_params_['alpha']

print(f"Best alpha (regularization strength): {best_alpha}")
print(f"Cross-validation score: {-lasso_grid.best_score_:.3f}")

# Make predictions
y_train_pred = lasso_model.predict(X_train_scaled)
y_val_pred = lasso_model.predict(X_val_scaled)

# Calculate performance metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Display results (focus on fit measures)
print(f"\nTraining Performance:")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE:  {train_mae:.3f}")
print(f"  R²:   {train_r2:.3f}")

print(f"\nValidation Performance:")
print(f"  RMSE: {val_rmse:.3f}")
print(f"  MAE:  {val_mae:.3f}")
print(f"  R²:   {val_r2:.3f}")

# Show feature selection results
non_zero_features = np.sum(lasso_model.coef_ != 0)
print(f"\nFeature Selection:")
print(f"  Features selected: {non_zero_features}/{len(predictor_cols_linear)}")
print(f"  Features eliminated: {len(predictor_cols_linear) - non_zero_features}")

# Coefficient magnitudes (features are scaled)
feature_importance = pd.DataFrame({
    'feature': predictor_cols_linear,
    'importance': np.abs(lasso_model.coef_)
})

print(f"\nTop 10 Most Important Features:")
print(feature_importance.head(10))


Best alpha (regularization strength): 0.1
Cross-validation score: 190.936

Training Performance:
  RMSE: 11.080
  MAE:  8.426
  R²:   0.525

Validation Performance:
  RMSE: 11.818
  MAE:  9.355
  R²:   0.482

Feature Selection:
  Features selected: 20/22
  Features eliminated: 2

Top 10 Most Important Features:
                feature  importance
0        fertility_rate    0.000000
1  fertility_adolescent    5.113766
2      urban_population    2.012054
3      dependency_ratio    6.478956
4       life_exp_female    6.474594
5      infant_mortality    2.404265
6      population_total    0.298316
7   secondary_enroll_fe    0.822782
8  gdp_per_capita_const    4.968650
9            gdp_growth    0.093619


### Ridge Regression (L2 regularization)

In [137]:
# Define alpha values to test (regularization strength)
alpha_values = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

# Initialize Ridge with GridSearch for hyperparameter tuning
ridge_grid = GridSearchCV(
    Ridge(random_state=42),
    param_grid={'alpha': alpha_values},
    cv=group_kfold,  # Use GroupKFold to prevent country leakage
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Fit on training data (using scaled data)
ridge_grid.fit(X_train_scaled, y_train, groups=groups_train)

# Get the best model
ridge_model = ridge_grid.best_estimator_
best_alpha = ridge_grid.best_params_['alpha']

print(f"Best alpha (regularization strength): {best_alpha}")
print(f"Cross-validation score: {-ridge_grid.best_score_:.3f}")

# Make predictions
y_train_pred = ridge_model.predict(X_train_scaled)
y_val_pred = ridge_model.predict(X_val_scaled)

# Calculate performance metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Display results
print(f"\nTraining Performance:")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE:  {train_mae:.3f}")
print(f"  R²:   {train_r2:.3f}")

print(f"\nValidation Performance:")
print(f"  RMSE: {val_rmse:.3f}")
print(f"  MAE:  {val_mae:.3f}")
print(f"  R²:   {val_r2:.3f}")

# Feature importance for Ridge (coefficient magnitudes)
feature_importance = pd.DataFrame({
    'feature': predictor_cols_linear,
    'importance': np.abs(ridge_model.coef_)
}).sort_values('importance', ascending=False)

print(f"\nTop 10 Most Important Features:")
print(feature_importance.head(10))

Best alpha (regularization strength): 100.0
Cross-validation score: 188.759

Training Performance:
  RMSE: 11.314
  MAE:  8.681
  R²:   0.505

Validation Performance:
  RMSE: 12.002
  MAE:  9.627
  R²:   0.466

Top 10 Most Important Features:
                 feature  importance
16            region_eap    9.537322
17            region_eca    9.487646
21            region_ssa    8.482274
18            region_lac    6.349557
4        life_exp_female    5.606925
1   fertility_adolescent    5.562440
8   gdp_per_capita_const    4.960477
3       dependency_ratio    4.561144
15      income_upper_mid    4.052803
13           income_high    4.021346


### Elastic Net Regression (L1+L2 regularization)

In [138]:
# Define grids for Elastic Net
alpha_values = [0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]
l1_ratios = [0.1, 0.3, 0.5, 0.7, 0.9]  # 0 → pure Ridge, 1 → pure Lasso

elastic_grid = GridSearchCV(
    ElasticNet(max_iter=5000, random_state=42),
    param_grid={'alpha': alpha_values, 'l1_ratio': l1_ratios},
    cv=group_kfold,                 # same GroupKFold by country
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

print("Fitting Elastic Net with hyperparameter tuning...")
elastic_grid.fit(X_train_scaled, y_train, groups=groups_train)

elastic_model = elastic_grid.best_estimator_
best_params = elastic_grid.best_params_

print(f"Best hyperparameters:")
print(f"  alpha: {best_params['alpha']}")
print(f"  l1_ratio: {best_params['l1_ratio']}")
print(f"Cross-validation score: {-elastic_grid.best_score_:.3f}")

# Predictions
y_train_pred = elastic_model.predict(X_train_scaled)
y_val_pred = elastic_model.predict(X_val_scaled)

# Metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"\nTraining Performance:")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE:  {train_mae:.3f}")
print(f"  R²:   {train_r2:.3f}")

print(f"\nValidation Performance:")
print(f"  RMSE: {val_rmse:.3f}")
print(f"  MAE:  {val_mae:.3f}")
print(f"  R²:   {val_r2:.3f}")

# Coefficient magnitudes
coef_importance = pd.DataFrame({
    'feature': predictor_cols_linear,
    'coef': elastic_model.coef_,
    'abs_coef': np.abs(elastic_model.coef_)
}).sort_values('abs_coef', ascending=False)

print("\nTop 10 features by |coefficient|:")
print(coef_importance.head(10))

Fitting Elastic Net with hyperparameter tuning...
Best hyperparameters:
  alpha: 0.1
  l1_ratio: 0.7
Cross-validation score: 188.273

Training Performance:
  RMSE: 11.364
  MAE:  8.715
  R²:   0.500

Validation Performance:
  RMSE: 11.945
  MAE:  9.571
  R²:   0.471

Top 10 features by |coefficient|:
                 feature      coef  abs_coef
17            region_eca  9.550699  9.550699
16            region_eap  9.487653  9.487653
21            region_ssa  8.562177  8.562177
18            region_lac  6.379608  6.379608
1   fertility_adolescent  5.172243  5.172243
4        life_exp_female -5.060523  5.060523
8   gdp_per_capita_const  4.683260  4.683260
3       dependency_ratio -4.455061  4.455061
15      income_upper_mid -3.672179  3.672179
13           income_high -3.506394  3.506394


## Support Vector Regression (SVR)

In [139]:
# Set simpler, regularized grid
param_grid = {
    # slightly favor smaller C (stronger regularization)
    'C': [0.01, 0.1, 1, 10],
    # a bit wider epsilon range (flatter function, less overfit)
    'epsilon': [0.05, 0.1, 0.2, 0.5],
    # avoid very small fixed gamma that can lead to overfitting
    'gamma': ['scale', 'auto']
}

svr_grid = GridSearchCV(
    SVR(kernel='rbf'),
    param_grid=param_grid,
    cv=group_kfold,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print("Fitting SVR (RBF) with more regularized hyperparameter grid...")
svr_grid.fit(X_train_scaled, y_train, groups=groups_train)
svr_model = svr_grid.best_estimator_
best_params = svr_grid.best_params_

print(f"Best hyperparameters:")
print(f"  C (regularization): {best_params['C']}")
print(f"  Gamma (kernel coef): {best_params['gamma']}")
print(f"  Epsilon (tolerance): {best_params['epsilon']}")
print(f"Cross-validation score: {-svr_grid.best_score_:.3f}")

# Make predictions
y_train_pred = svr_model.predict(X_train_scaled)
y_val_pred = svr_model.predict(X_val_scaled)

# Calculate performance metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Display results
print(f"\nTraining Performance:")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE:  {train_mae:.3f}")
print(f"  R²:   {train_r2:.3f}")

print(f"\nValidation Performance:")
print(f"  RMSE: {val_rmse:.3f}")
print(f"  MAE:  {val_mae:.3f}")
print(f"  R²:   {val_r2:.3f}")

print(f"\nModel Characteristics:")
print(f"  Kernel: RBF (Radial Basis Function)")
print(f"  Support vectors: {svr_model.n_support_}")
print(f"  Non-linear decision boundary")

Fitting SVR (RBF) with more regularized hyperparameter grid...
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best hyperparameters:
  C (regularization): 10
  Gamma (kernel coef): scale
  Epsilon (tolerance): 0.5
Cross-validation score: 180.817

Training Performance:
  RMSE: 5.711
  MAE:  3.286
  R²:   0.874

Validation Performance:
  RMSE: 11.103
  MAE:  8.636
  R²:   0.543

Model Characteristics:
  Kernel: RBF (Radial Basis Function)
  Support vectors: [2196]
  Non-linear decision boundary


## K-Nearest Neighbors Regression (KNN)

In [140]:
# Define hyperparameters to test
param_grid = {
    'n_neighbors': [3, 5, 7, 10, 15, 20],     # Number of neighbors
    'weights': ['uniform', 'distance'],        # Weighting scheme
    'metric': ['euclidean', 'manhattan']       # Distance metric
}

# Initialize KNN with GridSearch for hyperparameter tuning
knn_grid = GridSearchCV(
    KNeighborsRegressor(),
    param_grid=param_grid,
    cv=group_kfold,  # Use GroupKFold to prevent country leakage
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

print("Fitting KNN with hyperparameter tuning...")

# Fit on training data (using scaled data - KNN needs scaling!)
knn_grid.fit(X_train_scaled, y_train, groups=groups_train)

# Get the best model
knn_model = knn_grid.best_estimator_
best_params = knn_grid.best_params_

print(f"Best hyperparameters:")
print(f"  n_neighbors: {best_params['n_neighbors']}")
print(f"  weights: {best_params['weights']}")
print(f"  metric: {best_params['metric']}")
print(f"Cross-validation score: {-knn_grid.best_score_:.3f}")

# Make predictions
y_train_pred = knn_model.predict(X_train_scaled)
y_val_pred = knn_model.predict(X_val_scaled)

# Calculate performance metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Display results
print(f"\nTraining Performance:")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE:  {train_mae:.3f}")
print(f"  R²:   {train_r2:.3f}")

print(f"\nValidation Performance:")
print(f"  RMSE: {val_rmse:.3f}")
print(f"  MAE:  {val_mae:.3f}")
print(f"  R²:   {val_r2:.3f}")

print(f"\nModel Characteristics:")
print(f"  Non-parametric model")
print(f"  Memory-based learning")
print(f"  Local predictions based on nearest neighbors")

Fitting KNN with hyperparameter tuning...
Best hyperparameters:
  n_neighbors: 20
  weights: distance
  metric: euclidean
Cross-validation score: 208.827

Training Performance:
  RMSE: 0.000
  MAE:  0.000
  R²:   1.000

Validation Performance:
  RMSE: 13.788
  MAE:  10.402
  R²:   0.295

Model Characteristics:
  Non-parametric model
  Memory-based learning
  Local predictions based on nearest neighbors


## Tree-based models

### Random Forest

**Note**: Using reduced hyperparameter grids for initial model comparison. These smaller grids provide sufficient exploration to compare algorithm performance while keeping runtime manageable. Full hyperparameter optimization can be done later for the best-performing models.

In [141]:
# Note that we use the imputed but unscaled data for Random Forest

# Define hyperparameters to test (reduced grid for initial comparison)
param_grid = {
    'n_estimators': [100, 200],               # Number of trees
    'max_depth': [10, None],                  # Maximum tree depth
    'min_samples_split': [2, 5],              # Min samples to split node
    'min_samples_leaf': [1, 2],               # Min samples in leaf
    'max_features': ['sqrt', 0.3]             # Features per split
}

# Initialize Random Forest with GridSearch for hyperparameter tuning
rf_grid = GridSearchCV(
    RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    cv=group_kfold,  # Use GroupKFold to prevent country leakage
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1  # Show progress
)

print("Fitting Random Forest with hyperparameter tuning...")
print("This may take several minutes...")

# Fit on training data (using imputed but unscaled data)
rf_grid.fit(X_train_imputed, y_train, groups=groups_train)

# Get the best model
rf_model = rf_grid.best_estimator_
best_params = rf_grid.best_params_

print(f"Best hyperparameters:")
print(f"  n_estimators: {best_params['n_estimators']}")
print(f"  max_depth: {best_params['max_depth']}")
print(f"  min_samples_split: {best_params['min_samples_split']}")
print(f"  min_samples_leaf: {best_params['min_samples_leaf']}")
print(f"  max_features: {best_params['max_features']}")
print(f"Cross-validation score: {-rf_grid.best_score_:.3f}")

# Make predictions
y_train_pred = rf_model.predict(X_train_imputed)
y_val_pred = rf_model.predict(X_val_imputed)

# Calculate performance metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Display results
print(f"\nTraining Performance:")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE:  {train_mae:.3f}")
print(f"  R²:   {train_r2:.3f}")

print(f"\nValidation Performance:")
print(f"  RMSE: {val_rmse:.3f}")
print(f"  MAE:  {val_mae:.3f}")
print(f"  R²:   {val_r2:.3f}")

# Save feature importance for model interpretation and comparison
feature_importance = pd.DataFrame({
    'feature': predictor_cols_tree,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 10 Most Important Features:")
print(feature_importance.head(10))

Fitting Random Forest with hyperparameter tuning...
This may take several minutes...
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best hyperparameters:
  n_estimators: 100
  max_depth: 10
  min_samples_split: 5
  min_samples_leaf: 2
  max_features: sqrt
Cross-validation score: 227.948

Training Performance:
  RMSE: 4.208
  MAE:  3.098
  R²:   0.931

Validation Performance:
  RMSE: 13.403
  MAE:  10.706
  R²:   0.333

Top 10 Most Important Features:
                 feature  importance
7    secondary_enroll_fe    0.110851
8   gdp_per_capita_const    0.088424
6       population_total    0.084224
1   fertility_adolescent    0.081178
2       urban_population    0.080515
12           rule_of_law    0.077093
11          industry_gdp    0.073440
4        life_exp_female    0.070600
0         fertility_rate    0.067408
3       dependency_ratio    0.047004


### XGBoost

In [142]:
# Note that we use the raw unimputed data for XGBoost because it can handle missing values natively

# Clean column names to snake_case for consistency
X_train_raw_xgb = X_train_raw.copy()
X_val_raw_xgb = X_val_raw.copy()
X_test_raw_xgb = X_test_raw.copy()

# Convert to snake_case: lowercase, replace special chars and spaces with underscores
X_train_raw_xgb.columns = (X_train_raw_xgb.columns
                            .str.lower()
                            .str.replace('[^a-z0-9]+', '_', regex=True)
                            .str.strip('_'))
X_val_raw_xgb.columns = (X_val_raw_xgb.columns
                          .str.lower()
                          .str.replace('[^a-z0-9]+', '_', regex=True)
                          .str.strip('_'))
X_test_raw_xgb.columns = (X_test_raw_xgb.columns
                           .str.lower()
                           .str.replace('[^a-z0-9]+', '_', regex=True)
                           .str.strip('_'))

# Define hyperparameters to test (reduced grid for initial comparison)
param_grid = {
    'n_estimators': [100, 200],               # Number of boosting rounds
    'max_depth': [3, 6],                      # Maximum tree depth
    'learning_rate': [0.01, 0.1],             # Step size shrinkage
    'subsample': [0.9, 1.0],                  # Fraction of samples per tree
    'colsample_bytree': [0.9, 1.0]            # Fraction of features per tree
}

# Initialize XGBoost with GridSearch for hyperparameter tuning
xgb_grid = GridSearchCV(
    xgb.XGBRegressor(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    cv=group_kfold,  # Use GroupKFold to prevent country leakage
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1  # Show progress
)

print("Fitting XGBoost with hyperparameter tuning...")
print("This may take several minutes...")

# Fit on cleaned training data
xgb_grid.fit(X_train_raw_xgb, y_train, groups=groups_train)

# Get the best model
xgb_model = xgb_grid.best_estimator_
best_params = xgb_grid.best_params_

print(f"Best hyperparameters:")
print(f"  n_estimators: {best_params['n_estimators']}")
print(f"  max_depth: {best_params['max_depth']}")
print(f"  learning_rate: {best_params['learning_rate']}")
print(f"  subsample: {best_params['subsample']}")
print(f"  colsample_bytree: {best_params['colsample_bytree']}")
print(f"Cross-validation score: {-xgb_grid.best_score_:.3f}")

# Make predictions using cleaned data
y_train_pred = xgb_model.predict(X_train_raw_xgb)
y_val_pred = xgb_model.predict(X_val_raw_xgb)

# Calculate performance metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Display results
print(f"\nTraining Performance:")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE:  {train_mae:.3f}")
print(f"  R²:   {train_r2:.3f}")

print(f"\nValidation Performance:")
print(f"  RMSE: {val_rmse:.3f}")
print(f"  MAE:  {val_mae:.3f}")
print(f"  R²:   {val_r2:.3f}")

# Feature importance for XGBoost
feature_importance = pd.DataFrame({
    'feature': predictor_cols_tree,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 10 Most Important Features:")
print(feature_importance.head(10))

Fitting XGBoost with hyperparameter tuning...
This may take several minutes...
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best hyperparameters:
  n_estimators: 100
  max_depth: 6
  learning_rate: 0.01
  subsample: 0.9
  colsample_bytree: 0.9
Cross-validation score: 233.876

Training Performance:
  RMSE: 9.777
  MAE:  7.546
  R²:   0.630

Validation Performance:
  RMSE: 13.889
  MAE:  11.178
  R²:   0.284

Top 10 Most Important Features:
                 feature  importance
14            region_eap    0.095663
15            region_eca    0.095188
11          industry_gdp    0.090066
16            region_lac    0.081232
19            region_ssa    0.080724
13  income_level_encoded    0.072680
8   gdp_per_capita_const    0.064802
4        life_exp_female    0.063589
12           rule_of_law    0.059961
10          services_gdp    0.059404


### LightGBM

In [143]:
# Note that we use the raw unimputed data for LightGBM because it can handle missing values natively

# IMPORTANT: Clean column names for LightGBM (it doesn't support special characters)
# Convert to snake_case for consistency
X_train_raw_lgb = X_train_raw.copy()
X_val_raw_lgb = X_val_raw.copy()
X_test_raw_lgb = X_test_raw.copy()

# Convert to snake_case: lowercase, replace special chars and spaces with underscores
X_train_raw_lgb.columns = (X_train_raw_lgb.columns
                            .str.lower()
                            .str.replace('[^a-z0-9]+', '_', regex=True)
                            .str.strip('_'))
X_val_raw_lgb.columns = (X_val_raw_lgb.columns
                          .str.lower()
                          .str.replace('[^a-z0-9]+', '_', regex=True)
                          .str.strip('_'))
X_test_raw_lgb.columns = (X_test_raw_lgb.columns
                           .str.lower()
                           .str.replace('[^a-z0-9]+', '_', regex=True)
                           .str.strip('_'))

# Define hyperparameters to test (reduced grid for initial comparison)
param_grid = {
    'n_estimators': [100, 200],               # Number of boosting rounds
    'max_depth': [3, 6],                      # Maximum tree depth
    'learning_rate': [0.01, 0.1],             # Step size shrinkage
    'subsample': [0.9, 1.0],                  # Fraction of samples per tree
    'colsample_bytree': [0.9, 1.0],           # Fraction of features per tree
    'num_leaves': [31, 50]                    # Maximum number of leaves per tree
}

# Initialize LightGBM with GridSearch for hyperparameter tuning
lgb_grid = GridSearchCV(
    lgb.LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1),
    param_grid=param_grid,
    cv=group_kfold,  # Use GroupKFold to prevent country leakage
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1  # Show progress
)

print("Fitting LightGBM with hyperparameter tuning...")
print("This may take several minutes...")

# Fit on cleaned training data
lgb_grid.fit(X_train_raw_lgb, y_train, groups=groups_train)

# Get the best model
lgb_model = lgb_grid.best_estimator_
best_params = lgb_grid.best_params_

print(f"Best hyperparameters:")
print(f"  n_estimators: {best_params['n_estimators']}")
print(f"  max_depth: {best_params['max_depth']}")
print(f"  learning_rate: {best_params['learning_rate']}")
print(f"  subsample: {best_params['subsample']}")
print(f"  colsample_bytree: {best_params['colsample_bytree']}")
print(f"  num_leaves: {best_params['num_leaves']}")
print(f"Cross-validation score: {-lgb_grid.best_score_:.3f}")

# Make predictions using cleaned data
y_train_pred = lgb_model.predict(X_train_raw_lgb)
y_val_pred = lgb_model.predict(X_val_raw_lgb)

# Calculate performance metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Display results
print(f"\nTraining Performance:")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE:  {train_mae:.3f}")
print(f"  R²:   {train_r2:.3f}")

print(f"\nValidation Performance:")
print(f"  RMSE: {val_rmse:.3f}")
print(f"  MAE:  {val_mae:.3f}")
print(f"  R²:   {val_r2:.3f}")

# Feature importance for LightGBM
feature_importance = pd.DataFrame({
    'feature': predictor_cols_tree,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 10 Most Important Features:")
print(feature_importance.head(10))

Fitting LightGBM with hyperparameter tuning...
This may take several minutes...
Fitting 5 folds for each of 64 candidates, totalling 320 fits
Best hyperparameters:
  n_estimators: 100
  max_depth: 6
  learning_rate: 0.01
  subsample: 0.9
  colsample_bytree: 0.9
  num_leaves: 50
Cross-validation score: 233.796

Training Performance:
  RMSE: 9.932
  MAE:  7.662
  R²:   0.618

Validation Performance:
  RMSE: 14.503
  MAE:  11.536
  R²:   0.220

Top 10 Most Important Features:
                 feature  importance
6       population_total         441
2       urban_population         269
8   gdp_per_capita_const         263
12           rule_of_law         249
4        life_exp_female         210
0         fertility_rate         203
5       infant_mortality         186
1   fertility_adolescent         180
13  income_level_encoded         170
11          industry_gdp         147


### CatBoost


In [144]:
# Note that we use the raw unimputed data for CatBoost because it can handle missing values natively

# Clean column names to snake_case for consistency
X_train_raw_cat = X_train_raw.copy()
X_val_raw_cat = X_val_raw.copy()
X_test_raw_cat = X_test_raw.copy()

# Convert to snake_case: lowercase, replace special chars and spaces with underscores
X_train_raw_cat.columns = (X_train_raw_cat.columns
                            .str.lower()
                            .str.replace('[^a-z0-9]+', '_', regex=True)
                            .str.strip('_'))
X_val_raw_cat.columns = (X_val_raw_cat.columns
                          .str.lower()
                          .str.replace('[^a-z0-9]+', '_', regex=True)
                          .str.strip('_'))
X_test_raw_cat.columns = (X_test_raw_cat.columns
                           .str.lower()
                           .str.replace('[^a-z0-9]+', '_', regex=True)
                           .str.strip('_'))

# Define hyperparameters to test (reduced grid for initial comparison)
param_grid = {
    'iterations': [100, 200],                 # Number of boosting rounds
    'depth': [3, 6],                          # Maximum tree depth
    'learning_rate': [0.01, 0.1],             # Step size shrinkage
    'l2_leaf_reg': [1, 3, 5]                  # L2 regularization
}

# Initialize CatBoost with GridSearch for hyperparameter tuning
catboost_grid = GridSearchCV(
    CatBoostRegressor(random_state=42, verbose=0),
    param_grid=param_grid,
    cv=group_kfold,  # Use GroupKFold to prevent country leakage
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1  # Show progress
)

print("Fitting CatBoost with hyperparameter tuning...")
print("This may take several minutes...")

# Fit on cleaned training data
catboost_grid.fit(X_train_raw_cat, y_train, groups=groups_train)

# Get the best model
catboost_model = catboost_grid.best_estimator_
best_params = catboost_grid.best_params_

print(f"Best hyperparameters:")
print(f"  iterations: {best_params['iterations']}")
print(f"  depth: {best_params['depth']}")
print(f"  learning_rate: {best_params['learning_rate']}")
print(f"  l2_leaf_reg: {best_params['l2_leaf_reg']}")
print(f"Cross-validation score: {-catboost_grid.best_score_:.3f}")

# Make predictions using cleaned data
y_train_pred = catboost_model.predict(X_train_raw_cat)
y_val_pred = catboost_model.predict(X_val_raw_cat)

# Calculate performance metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)

train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

# Display results
print(f"\nTraining Performance:")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE:  {train_mae:.3f}")
print(f"  R²:   {train_r2:.3f}")

print(f"\nValidation Performance:")
print(f"  RMSE: {val_rmse:.3f}")
print(f"  MAE:  {val_mae:.3f}")
print(f"  R²:   {val_r2:.3f}")

# Feature importance for CatBoost
feature_importance = pd.DataFrame({
    'feature': predictor_cols_tree,
    'importance': catboost_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 10 Most Important Features:")
print(feature_importance.head(10))


Fitting CatBoost with hyperparameter tuning...
This may take several minutes...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best hyperparameters:
  iterations: 200
  depth: 6
  learning_rate: 0.01
  l2_leaf_reg: 5
Cross-validation score: 230.037

Training Performance:
  RMSE: 10.078
  MAE:  7.767
  R²:   0.607

Validation Performance:
  RMSE: 13.436
  MAE:  10.844
  R²:   0.330

Top 10 Most Important Features:
                 feature  importance
2       urban_population   12.195206
6       population_total   11.550263
1   fertility_adolescent    9.980830
13  income_level_encoded    9.464382
19            region_ssa    7.868200
11          industry_gdp    6.824441
10          services_gdp    6.376157
8   gdp_per_capita_const    6.018800
14            region_eap    5.950609
4        life_exp_female    5.287192


## Ensemble (ElasticNet + SVR)

### Try with 50/50 weights

In [145]:

# Individual model predictions
y_train_pred_elastic = elastic_model.predict(X_train_scaled)
y_val_pred_elastic   = elastic_model.predict(X_val_scaled)

y_train_pred_svr = svr_model.predict(X_train_scaled)
y_val_pred_svr   = svr_model.predict(X_val_scaled)

# Simple equal-weight average ensemble
w_elastic = 0.5
w_svr = 0.5

y_train_pred_ens = w_elastic * y_train_pred_elastic + w_svr * y_train_pred_svr
y_val_pred_ens   = w_elastic * y_val_pred_elastic   + w_svr * y_val_pred_svr

# Metrics: train
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred_ens))
train_mae  = mean_absolute_error(y_train, y_train_pred_ens)
train_r2   = r2_score(y_train, y_train_pred_ens)

# Metrics: validation
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_ens))
val_mae  = mean_absolute_error(y_val, y_val_pred_ens)
val_r2   = r2_score(y_val, y_val_pred_ens)

print("\nEnsemble (Elastic Net + SVR) - Training Performance:")
print(f"  RMSE: {train_rmse:.3f}")
print(f"  MAE:  {train_mae:.3f}")
print(f"  R²:   {train_r2:.3f}")

print("\nEnsemble (Elastic Net + SVR) - Validation Performance:")
print(f"  RMSE: {val_rmse:.3f}")
print(f"  MAE:  {val_mae:.3f}")
print(f"  R²:   {val_r2:.3f}")


Ensemble (Elastic Net + SVR) - Training Performance:
  RMSE: 7.826
  MAE:  5.733
  R²:   0.763

Ensemble (Elastic Net + SVR) - Validation Performance:
  RMSE: 10.826
  MAE:  8.382
  R²:   0.565


### Try tuning weights in a small grid

In [146]:
# Precomputed single-model preds
y_val_pred_elastic = elastic_model.predict(X_val_scaled)
y_val_pred_svr     = svr_model.predict(X_val_scaled)

best_w = None
best_r2 = -np.inf
best_stats = None

for w in np.linspace(0.0, 1.0, 21):  # 0.0, 0.05, ..., 1.0
    y_val_pred_ens = w * y_val_pred_elastic + (1 - w) * y_val_pred_svr
    r2  = r2_score(y_val, y_val_pred_ens)
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_ens))
    mae  = mean_absolute_error(y_val, y_val_pred_ens)
    if r2 > best_r2:
        best_r2 = r2
        best_w = w
        best_stats = (rmse, mae, r2)

print(f"Best weight on Elastic Net (1-w on SVR): w = {best_w:.2f}")
print(f"Validation RMSE: {best_stats[0]:.3f}, MAE: {best_stats[1]:.3f}, R²: {best_stats[2]:.3f}")

Best weight on Elastic Net (1-w on SVR): w = 0.35
Validation RMSE: 10.756, MAE: 8.351, R²: 0.571


## Model Comparison on the Validation Set

The table below summarizes validation performance for all candidate models on the country‑held‑out split, using the final, regularized RBF SVR grid (and omitting the linear‑kernel SVR). Metrics are RMSE, MAE, and R² on the validation set of unseen countries; lower RMSE/MAE and higher R² are better.

| **Model**            | **RMSE (val)** | **MAE (val)** | **R² (val)** |
|----------------------|----------------|----------------|--------------|
| OLS                  | 12.045         | 9.660          | 0.462        |
| Lasso                | 11.818         | 9.355          | 0.482        |
| Ridge                | 12.002         | 9.627          | 0.466        |
| Elastic Net          | 11.945         | 9.571          | 0.471        |
| SVR (RBF, tuned)     | 11.103         | 8.636          | 0.543        |
| KNN                  | 13.788         | 10.402         | 0.295        |
| Random Forest        | 13.403         | 10.706         | 0.333        |
| XGBoost              | 13.889         | 11.178         | 0.284        |
| LightGBM             | 14.503         | 11.536         | 0.220        |
| CatBoost             | 13.436         | 10.844         | 0.330        |
| **Ensemble (EN+SVR)**| **10.756**     | **8.351**      | **0.571**    |

Across the linear family, Lasso, Ridge, and Elastic Net all perform similarly, with validation R² in the 0.47–0.48 range and regularization smoothing out noise while preserving the main signal. The updated RBF SVR clearly outperforms these linear baselines (R² around 0.543), indicating that there is meaningful non‑linearity in the relationship between the predictors and FLFP that a flexible kernel can exploit. In contrast, KNN and all of the tree/boosting methods (Random Forest, XGBoost, LightGBM, CatBoost) achieve noticeably worse validation R² despite very strong training fits, which is consistent with overfitting country‑specific idiosyncrasies and failing to generalize to entirely new countries under the GroupKFold/country‑split evaluation.

Given this pattern, we treated SVR and Elastic Net as the two “good” and complementary models: SVR offering the strongest predictive accuracy, and Elastic Net offering a stable, interpretable linear structure. Rather than choosing a single winner, we constructed a simple ensemble that linearly combines their predictions and tuned the weight on Elastic Net using the validation set. Starting from a 50/50 mix (which already improved on both individual models), we searched over weights and found that giving 35 percent weight to Elastic Net and 65 percent to SVR yielded the best validation performance (RMSE about 10.756, R² about 0.571). This ensemble thus becomes the final model: it leverages the higher accuracy of SVR while tempering its extremes with the more conservative Elastic Net, improving generalization to new countries and aligning with the app’s goal of generating plausible predictions for hypothetical “slider‑defined” countries.

## Test Set Evaluation

We now evaluate all top-performing models on the held-out test set to assess their final generalization performance.


In [148]:
# Get predictions from all top models on test set
test_results = []

# OLS
y_test_pred_ols = ols_model.predict(X_test_scaled)
test_results.append({
    'Model': 'OLS',
    'RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred_ols)),
    'MAE': mean_absolute_error(y_test, y_test_pred_ols),
    'R²': r2_score(y_test, y_test_pred_ols)
})

# Lasso
y_test_pred_lasso = lasso_model.predict(X_test_scaled)
test_results.append({
    'Model': 'Lasso',
    'RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred_lasso)),
    'MAE': mean_absolute_error(y_test, y_test_pred_lasso),
    'R²': r2_score(y_test, y_test_pred_lasso)
})

# Ridge
y_test_pred_ridge = ridge_model.predict(X_test_scaled)
test_results.append({
    'Model': 'Ridge',
    'RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred_ridge)),
    'MAE': mean_absolute_error(y_test, y_test_pred_ridge),
    'R²': r2_score(y_test, y_test_pred_ridge)
})

# Elastic Net
y_test_pred_elastic = elastic_model.predict(X_test_scaled)
test_results.append({
    'Model': 'Elastic Net',
    'RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred_elastic)),
    'MAE': mean_absolute_error(y_test, y_test_pred_elastic),
    'R²': r2_score(y_test, y_test_pred_elastic)
})

# SVR
y_test_pred_svr = svr_model.predict(X_test_scaled)
test_results.append({
    'Model': 'SVR',
    'RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred_svr)),
    'MAE': mean_absolute_error(y_test, y_test_pred_svr),
    'R²': r2_score(y_test, y_test_pred_svr)
})

# Ensemble (35% Elastic Net + 65% SVR)
w_elastic = 0.35
w_svr = 0.65
y_test_pred_ens = w_elastic * y_test_pred_elastic + w_svr * y_test_pred_svr
test_results.append({
    'Model': 'Ensemble (EN+SVR)',
    'RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred_ens)),
    'MAE': mean_absolute_error(y_test, y_test_pred_ens),
    'R²': r2_score(y_test, y_test_pred_ens)
})

# Create DataFrame and sort by R² (descending)
test_df_results = pd.DataFrame(test_results).sort_values('R²', ascending=False)

print("Test Set Performance Comparison:")
display(test_df_results)

Test Set Performance Comparison:


,Model,RMSE,MAE,R²
5,Ensemble (EN+SVR),9.730482,7.830651,0.439785
4,SVR,9.833719,7.981249,0.427835
3,Elastic Net,11.648389,8.983109,0.197181
2,Ridge,11.808271,9.097965,0.174991
1,Lasso,12.412268,9.373027,0.088434
0,OLS,13.278586,9.947627,-0.043253


## Conclusion

On the held-out test set, the Ensemble (35% Elastic Net + 65% SVR) achieves the strongest performance with an R² of 0.440 and RMSE of 9.730, modestly outperforming SVR alone (R² = 0.428, RMSE = 9.834) and substantially outperforming all linear models. While test set performance is notably lower than validation performance across all models (likely reflecting distributional differences between the country splits), the ensemble demonstrates superior robustness: the regularized linear models show severe degradation on the test set (Elastic Net R² drops from 0.471 on validation to 0.197 on test, and OLS even produces negative R²), whereas the ensemble maintains reasonable predictive power. This pattern validates our modeling strategy of combining a flexible non-linear learner (SVR) with a stable linear baseline (Elastic Net)—the ensemble leverages SVR's superior accuracy while tempering its potential overfitting with the more conservative linear structure, resulting in the most reliable predictions for entirely new countries in the test set.